ok so in this notebook, i make acute lil embedder for all the big molecules. 
what you need:
- tmc1 inventory in selfies, already embedded
- datasets from pubchem in selfies

this script will ideally:
- embed full datasets with VICGAE
- drop any duplicates
- filter molecules for not hydrogen and contains anything we dont want from keep_elements

your outcome is a .csv of your recommendations, minus all the duplicates/weirdos like Te or something, and a .smi that you'll put through psi4 later for calculations. 
Note that this code is for SMILES and not SELFIES. 

In [1]:
#associated with VICGAE
from typing import Tuple, Dict
import warnings

from matplotlib import pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import joblib
from joblib import load, dump

import selfies as sf
from astrochem_embedding import VICGAE, get_paths, get_pretrained_path, Translator
import h5py
import dask
import pandas as pd
from periodictable import elements
from dask_ml.metrics import pairwise_distances
#from sklearn.metrics.pairwise import pairwise_distances
import dask.dataframe as dd
from tqdm.auto import tqdm
from common import encode_smiles, map_to_embeddings, bootstrap
import dask.array as da
import numpy.core.defchararray as np_f


In [2]:
#choose what elements you want to keep...
keep_elements = ["H", "C", "O", "N", "P" , "S", "Si"]
element_filter = [el.symbol for el in elements if el.symbol not in keep_elements]


In [3]:
#embedder, VICGAE, is already trained on a global set of molecules
#multipurpose
embedding_model = VICGAE.from_pretrained()


## load tmc1 inventory and matching regressor pickle
My inventory is already embedded

In [4]:
#this is embedded
#maybe... not needed
#detected_list = pd.read_hdf("tmc1_embedded.h5", "data")#tmc-1 detected inventory in embedded selfies


In [105]:
#vectorized tmc-1

dftmc1 = pd.read_csv(r'FI_updatedjan2025.csv')
#dftmc1 = pd.read_csv(r'OI_revisedjan2025.csv')
#dftmc1 = pd.read_csv(r'OD_updated_corrected.csv')
#dftmc1 = pd.read_csv(r'fd_UPDATED_jan2025.csv')


#dftmc1 = pd.read_csv(r'grouped-TMC1_full_Inventory_selfies_NOIsotopes_noCO.csv')
#grouped-TMC1_full_Inventory_selfies_NOIsotopes_noCO
#grouped-TMC1_ORIG_Inventory_selfies_NoIsotopes_NoCO.csv
#grouped-TMC1_ORIG_Inventory_selfies_WITHIsotopes_noCO
#grouped-TMC1_full_Inventory_selfies_WithIsotopes_noCO

detected_smiles_array = dftmc1['SMILES']
detected_selfies_array = dftmc1['SELFIES']
tmc1_selfies = dftmc1['SELFIES'].astype('str').apply(map_to_embeddings).values #from dataframe df, in selfies column, apply functionm aptoembeddings
vecs = np.vstack(tmc1_selfies) #this essentially makes the tensors into arrays, i think


### get GP regressor

In [107]:
gp_model = load("FI_updatedJan2025.pkl")
#gp_model = load("OD_updatedJan2025.pkl")

#gp_model = load("FD_updatedjan2025.pkl")



#gp_model = load("oct2024test-gpr-full-INVENTORY-Without-isotopes_5.pkl")
#gp_model = load("final-gpr-originventory-withisotopes-groupkfold-noCO.pkl")
#final-gpr-ORIG-INVENTORY-Without-isotopes.pkl
#final-gpr-full-INVENTORY-Without-isotopes.pkl
#final-gpr-fullinventory-withisotopes-groupkfold-noCO.pkl
#final-gpr-originventory-withisotopes-groupkfold-noCO.pkl

/Users/hannahts/miniconda3/envs/daskRec/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GaussianProcessRegressor from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## embed the full database set selfies


In [7]:
full_database = dd.read_csv(r'./collected_selfies.csv/*.part')
#full_database = dd.read_csv(r'./allSmilesIso_withSelfies.csv/*.part')
full_database_smiles_array = np.array(full_database['SMILES'])
full_database_selfies_array = np.array(full_database['SELFIES'])


# now, load the embeddings
i tested a few, so they're all here for history

In [8]:
#above was run by A for me,
#i'll load that here.

#dataset = np.loadtxt("selfies.txt") 
#dataset = np.load("/Users/hannahts/Dropbox (MIT)/embedded_selfies.npy", allow_pickle=True)
dataset = np.load("/Users/hannahts/Dropbox (MIT)/embedded_selfies_3Mdatasets.npy", allow_pickle=True)

In [311]:
#full_database7m = dd.read_csv(r'./allSmilesIso_withSelfies.csv/*.part')

In [9]:
dataset_dask=da.from_array(dataset, chunks=-1)
#dataset is embedded selfies

In [10]:
#this cell is really inefficnetly done but basically
#i am removing all instance of None, tu rning it into an array, stacking the array, stacked again (dumb but too lazy to change)
#then turning it into vectors

dataset_none_gone =  [(i if i is not None else  np.zeros_like(dataset[0])) for i in dataset]
new_dataset = np.array(dataset_none_gone) #np array
data2set = np.vstack(new_dataset)
data3set = np.vstack(data2set)


In [11]:
dataset_dask_noNone = da.from_array(data3set, chunks=-1) #vectors 

In [12]:
type(dataset_dask_noNone)

dask.array.core.Array

In [19]:
"""
this cell for smiles, for direct use of astrochem_embeddings to embed from smiles to embeddings
It SHOULD work for selfies as well, but this is not thoroughly tested...
"""

def is_not_pure_hydrogen(smi: str) -> bool:
    # this function checks if the molecule only contains
    # hydrogen, which stuffs up the obabel geometry generation
    
    try:
        #print("working", smi, type(smi))
        if not smi:
            return False
        characters = list(set([c for c in smi if c.isalpha()]))
#        characters = list(set([c for c in smi.tolist() if c.isalpha()]))
        if (len(characters) == 1) and (characters[0] == "H"):
            return False
        return True
    except:
        print("Not working", smi, f"{smi.shape=}", type(smi))

def is_not_complex(smi: str) -> bool:
    return "." not in smi
#    return "." not in smi.tolist()


# ##trying a new function
# def is_over_25(smi: str) -> bool:
#     if len(smi) > 25:
        


## finding indexes of molecules that are recommended by pairwise distance/distance in ml vector space to actual molecules


# getting recs

In [108]:
results = list()

for tmc_index, vector in tqdm(enumerate(vecs), total=len(vecs)):
    # we don't really care about the distance values, just the index
    distances = pairwise_distances(dataset_dask_noNone, vector[None,:]).ravel().compute()
    sorted_indices = np.argsort(distances) #ORIGINAL SCRIPT
    #sorted_indices = np.argsort(np.absolute(distances)) #testing
    
    
#     # get up to x number of recommendations, based on the :xxxx in sorted_indices

    top_ten = sorted_indices[:100] #100 recs PER molecule in the TMC-1 inventory
    #print(top_ten)
    
    for index in top_ten:



        if detected_smiles_array[tmc_index] != full_database_smiles_array[index]:
            results.append(
                {
                    "entry": index,
                    "SMILES_recommendation": full_database_smiles_array[index],
                    "SELFIES_recommendation": full_database_selfies_array[index],
                    "anchor": detected_smiles_array[tmc_index],
                    "distance": distances[index],
                }
            )
            
            

  0%|          | 0/157 [00:00<?, ?it/s]

## change results list into panda dataframe & filter as needed
first we drop duplicates
then we drop based on features in above cells--element filter, pure h, not complex

In [109]:
#put results into pandas
rec_df = pd.DataFrame(results)

In [110]:
rec_df

,entry,SMILES_recommendation,SELFIES_recommendation,anchor,distance
0,2251,C#CC#CCC#C,[C][#C][C][#C][C][C][#C],CC#CC#CC#C,0.007621
1,2285,C#CC#CC#CC,[C][#C][C][#C][C][#C][C],CC#CC#CC#C,0.022843
2,12462,CC#CC#CC#CC,[C][C][#C][C][#C][C][#C][C],CC#CC#CC#C,0.035413
3,461,C#CC#CCC,[C][#C][C][#C][C][C],CC#CC#CC#C,0.037292
4,12466,C#CC#CCC#CC,[C][#C][C][#C][C][C][#C][C],CC#CC#CC#C,0.037431
...,...,...,...,...,...
15637,3160023,[C-]#CC,[C-1][#C][C],[2H]CC#CC#C,0.094283
15638,940678,N#CCF,[N][#C][C][F],[2H]CC#CC#C,0.094669
15639,3160039,[C-]#CC#N,[C-1][#C][C][#N],[2H]CC#CC#C,0.094767
15640,2898669,CC#CN,[C][C][#C][N],[2H]CC#CC#C,0.094841


In [111]:
rec_df['SMILES_recommendation'] = rec_df.apply(lambda r: r['SMILES_recommendation'] if type(rec_df['SMILES_recommendation'])!=float else np.nan, axis=1)
rec_df.dropna(inplace=True)

In [112]:
rec_df['SMILES_recommendation'].isnull().values.any() #expect False

False

In [113]:

# first filter out non-unique suggests by their database index
#rec_df.drop_duplicates("entry", inplace=True)

# now filter out molecules that contain elements that are not in our keep list
rec_df = rec_df.loc[
    (~rec_df["SMILES_recommendation"].apply(lambda x: any([el in x for el in element_filter]))) &
    (rec_df["SMILES_recommendation"].apply(is_not_complex)) & # removes complexes
    (rec_df["SMILES_recommendation"].apply(is_not_pure_hydrogen)) # removes pure hydrogen
]
rec_df.reset_index(inplace=True, drop=True)

In [114]:
rec_df = rec_df.loc[(rec_df['SMILES_recommendation'].apply(lambda x: not "[nH]" in x))]
#rec_df = rec_df.loc[(rec_df['recommendation'].apply(lambda x: not "[nH]" in x.tolist()))]
#rec_df

## now, we should have a list of selfies strings.  let's predict their column densities!
# now we run predictions for the column densities


since i just want embeddings, i think i can just go straight into embeddings....

In [115]:
#SMILES
type(rec_df)
recommendations_arr = [x for x in rec_df['SMILES_recommendation'].to_numpy()]

ignored_list = []
ignored_indices = []
def safe_embeddings(smi):
    try:
        return embedding_model.embed_smiles(smi)
    except:
        ignored_list.append(smi)
        
        return []

In [65]:
"""
functionally the same as
#rec_vecs = np.vstack([embedding_model.embed_smiles(smi) for smi in rec_df["recommendation"]])

"""

# rec_vecs = list()
# for p in tqdm(range(len(rec_df))):
#     veci = safe_embeddings(recommendations_arr[p])
#     rec_vecs.append(veci)
    
    

'\nfunctionally the same as\n#rec_vecs = np.vstack([embedding_model.embed_smiles(smi) for smi in rec_df["recommendation"]])\n\n'

Ok, the next few lines should:
1. remove duplicates (final_arr)
2. vectorize/embed the array (final_vecs)
3. Convert the SMILES from final_arr into SELFIES (final_arr_selfies)
4. run embedded species through the model for cd
5. also create lists for the anchor molecule and distances


In [116]:
#removes elements from ignored_list from recommendations_arr
#also removes duplicates from inside recommendations_arr
final_arr=list( set(recommendations_arr)^set(ignored_list) )

In [117]:
final_vecs = list()
for p in tqdm(range(len(final_arr))):
    veca = safe_embeddings(final_arr[p])
    final_vecs.append(veca)

  0%|          | 0/1966 [00:00<?, ?it/s]

In [118]:
final_arr_selfies = []
for i in range(len(final_arr)):
    
    mol_ = sf.encoder(final_arr[i])
    final_arr_selfies.append(mol_)

In [119]:
#Making predictions
columns, std = [], []
for i in final_vecs:
    _c, _s = gp_model.predict(i, return_std=True)
    columns.append(_c)
    std.append(_s)

In [121]:
anchorlist, distancelist = [], []
for i in range(len(final_arr)):
    _a = rec_df['anchor'][i]
    _d = rec_df['distance'][i]
    anchorlist.append(_a)
    distancelist.append(_d)
    

# now you can combine all the information

In [122]:
final_full = np.column_stack([final_arr, final_arr_selfies, columns, std, anchorlist, distancelist])

In [123]:
#turn above numpy array into pandas for converting to csv below
final_rec_df = pd.DataFrame(final_full, columns = ['SMILES','SELFIES','COLUMN DENSITY', 'STD', 'anchor', 'distance'])

# make sure to change name of csv!!!!

In [125]:
#columns, std = gp_model.predict(rec_vecs, return_std=True)
final_rec_df["gpr_column"] = columns
final_rec_df["uncertainty"] = std
final_rec_df.to_csv("summary_FI_JAN2025.csv", index=False)



# make the .smi list

dont forget to change your file name

In [126]:
with open("targets-FI_jan2025.smi", "w+") as write_file:
    write_file.write("\n".join(final_rec_df["SMILES"].tolist()))
    
    #targets.smi is full-inventory-no-isotopes
    #targets-orig.smi original no isotope
    #targets-full.smi full no isotope